In [6]:
##Code in this file has been modified from Boris Murmann's github below, and the open-source textbook from Harald Pretl and colleagues:
# https://github.com/bmurmann/Book-on-gm-ID-design/blob/main/starter_files_open_source_tools/gf180mcuD/techsweep_plots_from_mat.ipynb
# Murmann and his colleague have written a book on gm/id-based design, "Systematic Design of Analog CMOS Circuits" (2017)

#Pretl's Open-Source Book: https://iic-jku.github.io/analog-circuit-design/analog_circuit_design.pdf

# Copyright 2024 Harald Pretl
# Licensed under the Apache License, Version 2.0 (the “License”); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at
# http://www.apache.org/licenses/LICENSE-2.0


import numpy as np
import scipy.constants as sc
import matplotlib.pyplot as plt
from pygmid import Lookup as lk

nfet = lk('nfet_03v3.mat')
pfet = lk('pfet_03v3.mat')
VDS1 = 1.65

In [35]:
# define the given parameters as taken from the specification table or inital guesses
c_load = 2e-12 #Expected given parasitics + breadboard mounting of device (from Xaiver)
gm_id_mirror = 6.5
gm_id_diff = 6.5
gm_id_amp = 8
gm_id_pop = 11
gm_id_nop= 8
#length in microns
l_mirror = 1
l_diff = 1
l_amp = 1
l_pop = 0.6
l_nop = 1

# Current Source Limitations
i_bias_in = 10e-6               #Reference current is 10uA. Note that if some internal design uses something like 30uA, you can mirror that value too
w_m6 = 3.5                      #UNIT NMOS WIDTH
l_56 = 1                        #UNIT NMOS LENGTH


#ignore for now
f_bw = 800e3 # -3dB bandwidth of the voltage buffer
i_total_limit = 50e-6
i_bias_in = 10e-6
output_voltage = 1.65
vin_min = 1
vin_max = 2.8
vdd_min = 3.2
vdd_max = 3.4

In [36]:
# we get the required gm of M1/2 from the -3dB bandwidth requirement of the voltage buffer
# we add a factor of 3 to allow for PVT variation plus additional MOSFET parasitic loading
gm_m12 = f_bw * 3 * 4*np.pi*c_load 
print('gm12 =', round(gm_m12/1e-3, 4), 'mS')

# since we know gm12 and the gmid we can calculate the bias current
id_m12 = gm_m12 / gm_id_m12
i_total = 2*id_m12
print('i_total (exact) =', round(i_total/1e-6, 1), 'µA')

# we round to 0.5µA bias currents
i_total = max(round(i_total / 1e-6 * 2) / 2 * 1e-6, 0.5e-6)
id_m12 = i_total/2
print('i_total (rounded) =', i_total/1e-6, 'µA')
if i_total < i_total_limit:
    print('[info] power consumption target is met!')
else:
    print('[info] power consumption target is NOT met!')


gm12 = 0.0603 mS


NameError: name 'gm_id_m12' is not defined

In [37]:
# Calculate gm values

#current current mirror output
I_m=i_bias_in * 2

#gm values since majority current is pulled to whichever transistor is on
gm_mirror = gm_id_mirror * I_m
gm_diff = gm_id_diff * I_m
gm_amp = gm_id_amp * I_m
gm_pop = gm_id_pop * I_m
gm_nop = gm_id_nop * I_m

#print the gm values for reference
print('gm_mirror', round(gm_mirror * 10e5, 1), 'uS')
print('gm_diff', round(gm_amp * 10e5,1), 'uS')
print('gm_amp', round(gm_amp * 10e5, 1), 'uS')
print('gm_pop', round(gm_pop * 10e5, 1), 'uS')
print('gm_nop', round(gm_nop * 10e5, 1), 'uS')


gm_mirror 130.0 uS
gm_diff 160.0 uS
gm_amp 160.0 uS
gm_pop 220.0 uS
gm_nop 160.0 uS


In [38]:
# we calculate the dc gain

#Intrinsic gain of circuit topologies

#pmos
gm_gds_mirror = pfet.lookup('GM_GDS', GM_ID=gm_id_mirror, L=l_mirror, VDS=VDS1, VSB=0)
gm_gds_diff = pfet.lookup('GM_GDS', GM_ID=gm_id_diff, L=l_diff, VDS=VDS1, VSB=0)
gm_gds_pop = pfet.lookup('GM_GDS', GM_ID=gm_id_pop, L=l_pop, VDS=VDS1, VSB=0)

print('gm_gds_mirror', gm_gds_mirror)
#nmos
gm_gds_amp = nfet.lookup('GM_GDS', GM_ID=gm_id_amp, L=l_amp, VDS=VDS1, VSB=0)
gm_gds_nop = nfet.lookup('GM_GDS', GM_ID=gm_id_nop, L=l_nop, VDS=VDS1, VSB=0)

#current mirror gds and intrinsic gain values 
gds_mirror = gm_mirror / gm_gds_mirror
print('a_mirror =', round(20*np.log10(gm_gds_mirror), 1), 'dB')

#diff pair gds and intrinsic gain values
gds_diff = gm_diff / gm_gds_diff
print('a_diff =', round(20*np.log10(gm_gds_diff), 1), 'dB')

#amp gds and intrinsic gain of amp & amp-diff topology
gds_amp = gm_amp / gm_gds_amp
print('a_amp =', round(20*np.log10(gm_gds_amp), 1), 'dB')
a_diff_amp = gm_diff/ (gds_amp + gds_diff)
print('a_diff_amp =', round(20*np.log10(a_diff_amp), 1), 'dB')

#output gain for gm/id expectation reference
print('a_nop =', round(20*np.log10(gm_gds_nop), 1), 'dB')
print('a_pop =', round(20*np.log10(gm_gds_pop), 1), 'dB')

#unitless gain for comparasion with techsweep plots
print('gm_gds_mirror', gm_gds_mirror)
print('gm_gds_diff', gm_gds_diff)
print('gm_gds_amp', gm_gds_amp)
print('gm_gds_nop', gm_gds_nop)
print('gm_gds_pop', gm_gds_pop)

gm_gds_mirror 448.05342823783684
a_mirror = 53.0 dB
a_diff = 53.0 dB
a_amp = 51.2 dB
a_diff_amp = 45.0 dB
a_nop = 51.2 dB
a_pop = 51.8 dB
gm_gds_mirror 448.05342823783684
gm_gds_diff 448.05342823783684
gm_gds_amp 363.41541415315965
gm_gds_nop 363.41541415315965
gm_gds_pop 388.1896364519071


In [39]:
# we calculate the MOSFET capacitance which adds to Cload, to see the impact on the BW

gm_cgs_diff = pfet.lookup('GM_CGS', GM_ID=gm_id_diff, L=l_diff, VDS=VDS1, VSB=0)
gm_cdd_diff = pfet.lookup('GM_CDD', GM_ID=gm_id_diff, L=l_diff, VDS=VDS1, VSB=0)
gm_cgs_amp = nfet.lookup('GM_CGS', GM_ID=gm_id_amp, L=l_amp, VDS=VDS1, VSB=0)
gm_cdd_amp = nfet.lookup('GM_CDD', GM_ID=gm_id_amp, L=l_amp, VDS=VDS1, VSB=0)
gm_cgs_nop = nfet.lookup('GM_CGS', GM_ID=gm_id_pop, L=l_pop, VDS=VDS1, VSB=0)
gm_cdd_nop = nfet.lookup('GM_CDD', GM_ID=gm_id_pop, L=l_pop, VDS=VDS1, VSB=0)

#Parasitic Capacitance
c_diff = abs(gm_diff/gm_cgs_diff) + abs(gm_diff/gm_cdd_diff)
c_amp = abs(gm_amp/gm_cdd_amp) + abs(gm_amp/gm_cgs_amp)
c_pop = abs(gm_pop/gm_cdd_nop) + abs(gm_amp/gm_cgs_nop)
c_load_parasitic = 1 / (1 / (c_diff + c_amp) + 1 / c_pop )

print('additional load capacitance =', round(c_load_parasitic/1e-15, 1), 'fF')

#ignore
f_bw = gm_nop / (4*np.pi * (c_load + c_load_parasitic))
print('unity gain bandwidth incl. parasitics =', round(f_bw/1e6, 2), 'MHz')

additional load capacitance = 10.6 fF
unity gain bandwidth incl. parasitics = 6.33 MHz


In [40]:
#Calculate Vgs

vgs_mirror = pfet.look_upVGS(GM_ID=gm_id_mirror, L=l_mirror, VDS=VDS1, VSB=0.0)
vgs_diff = pfet.look_upVGS(GM_ID=gm_id_diff, L=l_diff, VDS=VDS1, VSB=0.0)
vgs_amp = nfet.look_upVGS(GM_ID=gm_id_amp, L=l_amp, VDS=VDS1, VSB=0.0)
vgs_nop = nfet.look_upVGS(GM_ID=gm_id_nop, L=l_nop, VDS=VDS1, VSB=0.0)
vgs_pop = pfet.look_upVGS(GM_ID=gm_id_pop, L=l_pop, VDS=VDS1, VSB=0.0)

#Print the values of Vgs
print('vgs_mirror =', round(float(vgs_mirror), 3), 'V')
print('vgs_diff =', round(float(vgs_diff), 3), 'V')
print('vgs_amp =', round(float(vgs_amp), 3), 'V')
print('vgs_nop =', round(float(vgs_nop), 3), 'V')
print('vgs_pop =', round(float(vgs_pop), 3), 'V')



vgs_mirror = 1.058 V
vgs_diff = 1.058 V
vgs_amp = 0.889 V
vgs_nop = 0.889 V
vgs_pop = 0.923 V


In [41]:
# calculate settling time due to slewing with the calculated bias current
vout = 3.3
t_slew = (c_load + c_load_parasitic) * vout / I_m
print('slewing time =', round(t_slew/1e-6, 3), 'µs')
t_settle = 5/(2*np.pi*f_bw) #ignore
print('settling time =', round(t_settle/1e-6, 3), 'µs')

slewing time = 0.332 µs
settling time = 0.126 µs


In [42]:
# calculate total rms output noise
sth_m12 = nfet.lookup('STH_GM', VGS=vgs_m12, L=l_12, VDS=VDS1, VSB=0) * gm_m12
gamma_m12 = sth_m12/(4*1.38e-23*300*gm_m12)
sth_m34 = pfet.lookup('STH_GM', VGS=vgs_m34, L=l_34, VDS=VDS1, VSB=0) * gm_m34
gamma_m34 = sth_m34/(4*1.38e-23*300*gm_m34)
output_noise_rms = np.sqrt(1.38e-23*300 / (c_load + c_load_parasitic) * (2*gamma_m12 + 2*gamma_m34*gm_m34/gm_m12))
print('output noise =', round(output_noise_rms/1e-6, 1), 'µVrms')


NameError: name 'vgs_m12' is not defined

In [43]:
# calculate all widths

#Current Mirror Width
id_w_mirror = pfet.lookup('ID_W', GM_ID=gm_id_mirror, L=l_mirror, VDS=vgs_mirror, VSB=0)
w_mirror = I_m / id_w_mirror
print ('w_mirror =', round(w_mirror, 2))

#Diff Pair Width
id_w_diff = pfet.lookup('ID_W', GM_ID=gm_id_diff, L=l_diff, VDS=vgs_diff, VSB=0)
w_diff = I_m / id_w_diff
print ('w_diff =', round(w_diff, 2))

#Amp Width
id_w_amp = nfet.lookup('ID_W', GM_ID=gm_id_amp, L=l_amp, VDS=vgs_amp, VSB=0)
w_amp = I_m / id_w_amp
print ('w_amp =', round(w_amp, 2))

#Hystersis
w_hystersis = w_amp * 2 #decent rule of thumb to ensure that the current is larger than amp fets
print('w_hystersis =', round(w_hystersis, 2))

#Output Transistor Width
id_w_nop = nfet.lookup('ID_W', GM_ID=gm_id_nop, L=l_nop, VDS=vgs_nop, VSB=0)
w_nop = I_m / id_w_nop
print ('w_nop =', round(w_nop, 2))

id_w_pop = pfet.lookup('ID_W', GM_ID=gm_id_pop, L=l_pop, VDS=vgs_pop, VSB=0)
w_pop = I_m / id_w_pop
print ('w_pop =', round(w_pop, 1))


w_mirror = 15.19
w_diff = 15.19
w_amp = 5.46
w_hystersis = 10.93
w_nop = 5.46
w_pop = 26.2
